In [2]:
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import undetected_chromedriver as uc
import os

In [6]:
# URL of the ShareChat page
url = "https://sharechat.com/trending/Hindi"

chrome_options = uc.ChromeOptions()
chrome_options.add_argument('--disable-popup-blocking')
driver = uc.Chrome(options=chrome_options)
driver.get(url)

if not os.path.exists('./imgs'):
    os.makedirs('./imgs')

scroller = driver.find_element(By.XPATH,
                               "//div[@class='infinite-list-wrapper']")

posts = scroller.find_elements(
    By.XPATH, './/div[@data-cy="image-post"]')  # | //div[@data-cy="video-post"] | //div[@data-cy="gif-post"]')
print(len(posts))
# time.sleep(10)

for post in posts:
    try:
        # get the data-post-ph attribute of post
        post_ph = post.get_attribute('data-post-ph')
        print("Post PH:", post_ph)
        # Locate the elements containing the information
        author_element = post.find_element(
            By.CSS_SELECTOR, 'strong[data-cy="author-name"]')
        author_name = author_element.text

        imgElement = post.find_element(
            By.XPATH, './/div[contains(@class, "Pos(a)") and contains(@class, "Start(0)") and contains(@class, "W(100%)") and contains(@class, "T(0)") and contains(@class, "H(100%)")]')

        # driver.execute_script("arguments[0].scrollIntoView();", imgElement)
        # time.sleep(1)

        # screenshot
        # check if image already exists and add a number to it
        num = 0
        while os.path.exists(f'./imgs/{author_element.text}_{num}.png'):
            num += 1

        # scroll into view
        driver.execute_script("arguments[0].scrollIntoView();", author_element)

        time.sleep(3)
        print("Taking screenshot")
        imgElement.screenshot(
            f'./imgs/{author_element.text}_{num}.png')

        # div[@class="H(100%) Pstart($xs) Fxg(1) Miw(0)"]
        topDetailsDiv = post.find_element(
            By.XPATH, './/div[@class="H(100%) Pstart($xs) Fxg(1) Miw(0)"]')

        # find direct div inside it
        topDetailsDiv = topDetailsDiv.find_element(
            By.XPATH, './div[@class="H(100%) Ta(start) D(f) Jc(c) Ai(fs) Fxd(c)"]')

        # direct divs inside it
        innerDivs = topDetailsDiv.find_elements(
            By.XPATH, './div')
        text = innerDivs[1].text.split('•')
        years_before = text[1].strip()
        number_of_views = text[0].strip()

        post_caption = post.find_element(
            By.XPATH, './/div[@data-cy="post-caption"]')

        print("Author Name:", author_name)
        print("Number of Views:", number_of_views)
        print("Years Before:", years_before)
        print("Post Caption:", post_caption.text)

        # open new page comments
        commentLink = f"https://sharechat.com/comment/{post_ph}"
        print("Number of windows:", len(driver.window_handles))

        original_window = driver.current_window_handle

        driver.execute_script(f"window.open('{commentLink}');")
        time.sleep(1)

        new_window = [
            window for window in driver.window_handles if window != original_window][0]
        driver.switch_to.window(new_window)
        time.sleep(1)

        topBar = driver.find_element(
            By.XPATH, '//ul[@class="List(n)  D(f) Ai(c) W(100%) H(100%) "]')
        # click on 3rd li
        toFindinLike = "लाइक"
        idx = 0

        topEls = topBar.find_elements(By.XPATH, './li')
        for idx, li in enumerate(topEls):
            if toFindinLike in li.text:
                break

        likeCount = int(topEls[idx].text.replace(toFindinLike, '').strip())
        print("Likes: ", likeCount)
        # click
        topEls[idx].click()

        mainDiv = driver.find_element(
            By.XPATH, '//div[@class="Ovy(a) Fxg(1) W(100%) Maw(600px) M(a)"]')

        # loadedlikes = number of a
        loaded_likes = len(mainDiv.find_elements(
            By.XPATH, './/a[@data-cy="avatar-tag"]'))
        pbar = tqdm(total=likeCount)
        scroll_increment = 200  # The amount by which to increment the scroll each time
        current_scroll_position = 0  # Keep track of the current scroll position

        while loaded_likes < likeCount:
            driver.execute_script(
                f"arguments[0].scrollTop = {current_scroll_position}", mainDiv)
            current_scroll_position += scroll_increment
            listEls = mainDiv.find_elements(
                By.XPATH, './/a[@data-cy="avatar-tag"]')
            loaded_likes = len(listEls)

            pbar.update(loaded_likes - pbar.n)

        pbar.close()
        time.sleep(3)

        # get the list of all the users
        listEls = mainDiv.find_elements(
            By.XPATH, './/a[@data-cy="avatar-tag"]')

        # in this, strong tag with data-cy="author-name" is the name of the user, get it
        # and add it to a list
        users = []
        for li in listEls:
            users.append(li.find_element(
                By.XPATH, './/strong[@data-cy="author-name"]').text)

        print("Users:", users)
        print("Number of users:", len(users))
        driver.close()
        driver.switch_to.window(original_window)

    except Exception as e:
        print("An error occurred:", str(e))
        continue

7
Post PH: 1R8P0wm7
Taking screenshot
Author Name: ⚔️(♡pooja♡)⚔️
Number of Views: 56K ने देखा
Years Before: 2 दिन पहले
Post Caption: #🙏महाकाल स्टेटस #🔱हर हर महादेव #🕉 ओम नमः शिवाय 🔱 ##🙏महाकाल स्टेटस #🔱हर हर महादेव #🕉 ओम नमः शिवाय 🔱 #📝महाकाल भक्ति स्टेटस🙏 #🙏जय शिव शम्भू #🔱🛕महाकाल भक्त 🛕🔱
Number of windows: 1
Likes:  2537


  0%|          | 0/194 [00:25<?, ?it/s]


An error occurred: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=116.0.5845.179)
Stacktrace:
	GetHandleVerifier [0x003537C3+48947]
	(No symbol) [0x002E8551]
	(No symbol) [0x001EC92D]
	(No symbol) [0x001D36E5]
	(No symbol) [0x00238CDB]
	(No symbol) [0x00247113]
	(No symbol) [0x00234E86]
	(No symbol) [0x002116C7]
	(No symbol) [0x0021284D]
	GetHandleVerifier [0x0059FDF9+2458985]
	GetHandleVerifier [0x005E744F+2751423]
	GetHandleVerifier [0x005E1361+2726609]
	GetHandleVerifier [0x003D0680+560624]
	(No symbol) [0x002F238C]
	(No symbol) [0x002EE268]
	(No symbol) [0x002EE392]
	(No symbol) [0x002E10B7]
	BaseThreadInitThunk [0x76D600C9+25]
	RtlGetAppContainerNamedObjectPath [0x773A7B1E+286]
	RtlGetAppContainerNamedObjectPath [0x773A7AEE+238]

An error occurred: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=116.0.5845.179)
Stacktrace:
	GetHandleVerifier [0

In [ ]:
# clear imgs
for filename in os.listdir('./imgs'):
    os.remove(f'./imgs/{filename}')

NameError: name 'os' is not defined